TODO: 
- ensure h5py is installed
- save progress too,to h5 file
- see why sklearn model saving/restore is failing

- for a collection of timeseries
    - compute target feature(s)
    - split each to train/dev/test
  
    - compute performance of ma & ewma predictors. score in terms of rms error as %.??
    - compute performance of RF predictor
    - ... performance of nn net windowed predictor
    - ... performance of rnn predictor. (different architectures).
    
- apply to both stock, and FX time series, for comparison.


DataSets:
- name
- px
- target 
- features.. - model specific.
ds{
- name
- X
- Y
}

data {
    name: ds
    }

In [1]:
from utils import load_ds1
from IPython.display import display as disp

In [2]:
data = load_ds1()

['AAPL',
 'ABT',
 'AIG',
 'AMGN',
 'APA',
 'APC',
 'AXP',
 'BA',
 'BAC',
 'BAX',
 'BIIB',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'CMCSA',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'CVX',
 'DD',
 'DE',
 'DELL',
 'DHR',
 'DIS',
 'DOW',
 'EMC',
 'EMR',
 'EOG',
 'EXC',
 'F',
 'GE',
 'GILD',
 'HAL',
 'HD',
 'HON',
 'HPQ',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'LLY',
 'LOW',
 'MCD',
 'MDT',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NKE',
 'ORCL',
 'OXY',
 'PEP',
 'PFE',
 'PG',
 'PNC',
 'PX',
 'QCOM',
 'SBUX',
 'SLB',
 'SO',
 'SP500',
 'SPG',
 'T',
 'TGT',
 'TJX',
 'TWX',
 'TXN',
 'UNH',
 'UNP',
 'USB',
 'UTX',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [11]:
ds = data['CL']
disp("X_train", ds.X_train.head(3), ds.X_train.tail(3))
disp("Y_train", ds.Y_train.head(3), ds.Y_train.tail(3))

'X_train'

,lret-1,lret-2,lret-3,lret-4,lret-5,lret-6,lret-7,lret-8,lret-9,lret-10,...,lret-31,lret-32,lret-33,lret-34,lret-35,lret-36,lret-37,lret-38,lret-39,lret-40
1990-02-28,0.006631,0.024528,0.026790,0.020025,0.013305,0.022274,0.008853,-0.004394,0.006631,-0.002199,...,-0.081056,-0.070925,-0.085079,-0.070925,-0.074989,-0.062745,-0.060690,-0.095071,-0.104962,-0.108892
1990-03-01,-0.017778,-0.011148,0.006750,0.009012,0.002247,-0.004473,0.004496,-0.008926,-0.022172,-0.011148,...,-0.084620,-0.098835,-0.088703,-0.102857,-0.088703,-0.092767,-0.080523,-0.078469,-0.112850,-0.122740
1990-03-02,-0.004496,-0.022274,-0.015644,0.002254,0.004516,-0.002249,-0.008969,0.000000,-0.013421,-0.026668,...,-0.085019,-0.089116,-0.103330,-0.093199,-0.107353,-0.093199,-0.097263,-0.085019,-0.082965,-0.117345


,lret-1,lret-2,lret-3,lret-4,lret-5,lret-6,lret-7,lret-8,lret-9,lret-10,...,lret-31,lret-32,lret-33,lret-34,lret-35,lret-36,lret-37,lret-38,lret-39,lret-40
2009-02-02,-0.010821,-0.013585,0.007645,0.033505,0.040602,0.042222,0.027896,0.020571,0.034382,0.018838,...,0.033423,0.086213,0.089248,0.100456,0.090093,0.076496,0.053424,0.027214,0.047252,0.017265
2009-02-03,0.021830,0.011009,0.008246,0.029475,0.055335,0.062432,0.064052,0.049726,0.042401,0.056212,...,0.062305,0.055253,0.108044,0.111079,0.122286,0.111923,0.098327,0.075254,0.049045,0.069082
2009-02-04,-0.026192,-0.004361,-0.015182,-0.017946,0.003284,0.029144,0.036240,0.037860,0.023534,0.016209,...,0.008514,0.036113,0.029062,0.081852,0.084887,0.096094,0.085732,0.072135,0.049062,0.022853


'Y_train'

,target-1
1990-02-28,-0.017778
1990-03-01,-0.004496
1990-03-02,-0.011325


,target-1
2009-02-02,0.021830
2009-02-03,-0.026192
2009-02-04,0.005604


In [ ]:
def logreturn_features_and_targets(df, return_lookbacks=[1], target_lookaheads=[1]):
    results = pd.DataFrame(index=df.index)  
    feature_cols = []
    target_cols = []
    
    for lb in return_lookbacks:
        col = 'lret-' + str(lb)
        results[col] = logreturn(df['px'], df['px'].shift(lb))
        feature_cols.append(col)       
    
    # add target feature to predict
    for la in target_lookaheads:
        col = 'target-' + str(la)
        results[col] = logreturn(df['px'].shift(-la), df['px'])
        target_cols.append(col)
        
    results = results.dropna()
    
    return results[feature_cols], results[target_cols]

In [ ]:

def load_ds():
    features_and_targets = partial(logreturn_features_and_targets, return_lookbacks=np.arange(40)+1, target_lookaheads=[1])
    return load_stock_datasets(features_and_targets)